## [ 1. Python으로 MongoDB 제어하기 ]

### (1) pymongo 라이브러리 설치
> pip install pymongo

<br>
<br>

### (2) Mongodb 접속
-----
>import pymongo
>객체변수 = pymongo.MongoClient(mongo_server. 27017)

- 따로, 서버 ip를 지정하지 않을 경우 사용하는 컴퓨터의 로컬ip가 들어간다.
    - ex) connection = pymongo.MongoClient()



- AWS EC2 서버에서 접속을 원할 경우, 'mongodb://AWS_EC2_IP' 를 ()에 넣으면된다.
    - ex) connection = pymongo.MongoClient('mongodb://15.164.160.186')

In [1]:
import pymongo
connection = pymongo.MongoClient()

<br>
<br>

### (3) Database 생성/연결
-----
- conn_db 변수에 test 데이터베이스의 객체가 들어간다.
- test라는 데이터베이스가 해당 ip에 없을경우 생성된다.

In [2]:
conn_db = connection.test

In [ ]:
conn_db = connection['test'] # 이렇게도 선언 가능하다.

-----
- print를 사용하여 database가 저장된 객체를 출력하면, 출력된 MongoClient함수를 통해서 해당 database가 저장된 host가 어딘지 확인 할 수 있다.

In [3]:
print(conn_db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')


In [8]:
print(dir(conn_db)) 

['_BaseObject__codec_options', '_BaseObject__read_concern', '_BaseObject__read_preference', '_BaseObject__write_concern', '_Database__client', '_Database__incoming_copying_manipulators', '_Database__incoming_manipulators', '_Database__name', '_Database__outgoing_copying_manipulators', '_Database__outgoing_manipulators', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply_incoming_copying_manipulators', '_apply_incoming_manipulators', '_command', '_create_or_update_user', '_current_op', '_default_role', '_fix_incoming', '_fix_outgoing', '_list_collections', '_read_preference_for', '_retryable_read_command', '_write_concern_for'

----
- name을 사용하여 해당 데이터베이스의 이름이 확인 가능하다.

In [9]:
print(conn_db.name)

test


<br>
<br>

### (4) Database의 Collection 사용하기

----
- test 데이터베이스 안에 있는 test_collection의 객체가 conn_db_collection 변수에 저장된다.

In [4]:
conn_db_collection = conn_db.test_collection

In [ ]:
conn_db_collection = conn_db['test_collection'] # 이렇게도 쓸 수 있다.

----
- 출력된 MongoClient함수를 통해서 해당 collection이 저장된 host, 소속 Database, 그리고 collection name을 확인 할 수 있다.

In [13]:
conn_db_collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'test_collection')

#### 주의) database에 데이터를 저장하기 전까지는 robo3T에서 직접적으로 생성된 database나 collection을 확인 할 수 없다.

## [ 2. Python으로 MongoDB CRUD 제어하기 ]

### (1) Document Insert(삽입) ( insert_one() 과 insert_many() )
----
- `insert_one()`
    - mongodb shell 명령어 : `insertOne()`

In [15]:
insert_data = {
        'Name' : 'JihyunRyu',
        'Age' : 23,
        'School' : 'SeoulTech.Univ',
        'Major' : 'MSDE'
}

In [16]:
conn_db_collection.insert_one(insert_data) # 아래 출력 내용은 데이터가 잘 저장되었다는 이야기

----
- refresh 후 정상적으로 생성 및 저장되어 있음을 확인 할 수 있다.


![20](https://user-images.githubusercontent.com/53929665/90895814-a7394200-e3fd-11ea-9f89-61406e010aeb.JPG)

----
- `insert_many` 를 사용하여 (여러개) list of Document 생성할 수 있다. 
- `[ ]`를 사용하는 것을 항상 염두!

In [17]:
conn_db_collection.insert_many(
    [
        {'Student ID' : '17102020', 'Grade' : 'Senior'},
        {'Github URL' : 'https://github.com/jhryu1208', 'Github User_ID' : 'jhryu1208'}
    ]
)

---
- refresh 후 정상적으로 생성 및 저장되어 있음을 확인 할 수 있다.


![21](https://user-images.githubusercontent.com/53929665/90897749-8f16f200-e400-11ea-9bf3-c00a1071e8dd.JPG)


<br>
<br>

### (2) 참조
#### < Document INSERT 후,  _id(primary key)를 확인하는 방법 >

In [18]:
email = {'Naver' : 'jhryu1208@naver.com', 'Google' : 'jhryu1208@gmail.com'}

----
- INSERT된 정보가 email_id 변수에 들어간다.

In [19]:
email_id = conn_db_collection.insert_one(email)

----
- INSERT가 성공적으로 되었음을 확인할 수 있다.

In [20]:
email_id

----
- `inserted_id`를 사용하여 추가된 데이터의 `primary key`를 확인 할 수 있다.

![22](https://user-images.githubusercontent.com/53929665/90961344-1fc5fe80-e4e3-11ea-9207-a433c17d1fde.JPG)


In [21]:
email_id.inserted_id

ObjectId('5f414c1684fc9297a70c5c1b')

#### < `estimated_document_count()` 메서드는 컬렉션 객체와 함께 쓰여서 총 Document수를 알려준다. >
    - count_documents({})
    - count() 함수는 최신 pymongo 라이브러리에서는 사용 권장되지 않는다.

![23](https://user-images.githubusercontent.com/53929665/90961392-89dea380-e4e3-11ea-9907-f9bee6a133b9.JPG)

In [22]:
conn_db_collection.count_documents({}) # 위와 동일하게 Document수가 4개인 것을 확인 할 수 있다.

4

---
- <b>list</b>와 <b>dictionary</b>를 활용하여 insert하기

In [23]:
# 리스트, 객체 삽입 가능
# 총 2개의 document 추가
conn_db_collection.insert_one({'title' : '암살', 'castings' : ['이정재', '전지현', '하정우']})
conn_db_collection.insert_one(
    {
        'title' : '실미도', 
        'castings' : ['설경구', '안성기'], 
        'datetime' : 
        {
            'year' : '2003', 
            'month' : 3,
            'val' : 
            {
                'a' :
                {
                    'b' : 1
                }
            }
        }
    }
)

In [25]:
# 총 7개의 document 추가
data = list()                                    # 리스트 형태의 자료형을 만들어서
data.append({'name' : 'aaron', 'age' : 20})      # append함수를 통해 json데이터를 추가할 수있다.
data.append({'name' : 'bob', 'age' : 30})
data.append({'name' : 'cathy', 'age' : 25})
data.append({'name' : 'david', 'age' : 27})
data.append({'name' : 'erick', 'age' : 28})
data.append({'name' : 'fox', 'age' : 32})
data.append({'name' : 'hmm'})

conn_db_collection.insert_many(data)

In [27]:
# total : 4 + 2 + 7 = 13개
conn_db_collection.count_documents({})

13

<br>
<br>

### (2) Document Read(읽기) ( find_one() 과 find() )

In [6]:
import pymongo
connection = pymongo.MongoClient()
conn_db = connection.test
conn_db_collection = conn_db.test_collection

----
- <b>`find_one()`</b> : 가장 빨리 검색되는 document 검색하기
    - collection의 최상단 document가 검색된다.

In [7]:
conn_db_collection.find_one()

{'_id': ObjectId('5f3fcaf084fc9297a70c5c18'),
 'Name': 'JihyunRyu',
 'Age': 23,
 'School': 'SeoulTech.Univ',
 'Major': 'MSDE'}

---
- `find_one()` 안에 조건을 넣을 때는 사전 형식 `{key : value}`으로 해야한다.
- 조건을 붙여 검색할 경우, 최상단 부터 아래로 내려오면서 조건에 부합한 document를 출력한다.

In [13]:
jihyun = conn_db_collection.find_one( {'age' : 20})
jihyun

{'_id': ObjectId('5f414e5b84fc9297a70c5c1e'), 'name': 'aaron', 'age': 20}

---
- <b>`find()`</b> : 검색되는 <U>모든</U> Document 읽어오기
    - find 메서드를 이용하면, 반환값은 리스트 형태이다.

In [14]:
docs = conn_db_collection.find()

In [15]:
for doc in docs:
    print(doc)

{'_id': ObjectId('5f3fcaf084fc9297a70c5c18'), 'Name': 'JihyunRyu', 'Age': 23, 'School': 'SeoulTech.Univ', 'Major': 'MSDE'}
{'_id': ObjectId('5f3fcde484fc9297a70c5c19'), 'Student ID': '17102020', 'Grade': 'Senior'}
{'_id': ObjectId('5f3fcde484fc9297a70c5c1a'), 'Github URL': 'https://github.com/jhryu1208', 'Github User_ID': 'jhryu1208'}
{'_id': ObjectId('5f414c1684fc9297a70c5c1b'), 'Naver': 'jhryu1208@naver.com', 'Google': 'jhryu1208@gmail.com'}
{'_id': ObjectId('5f414e2b84fc9297a70c5c1c'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('5f414e2b84fc9297a70c5c1d'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('5f414e5b84fc9297a70c5c1e'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('5f414e5b84fc9297a70c5c1f'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('5f414e5b84fc9297a70c5c20'), 'name': 'cathy', 'age': 25}
{'_id': ObjectId('5f414e5b84fc9297a70c5c21'), 'name': 'david', 'age': 27}
{'_id'

---
- `find_one()` 과 마찬가지로, `find()` 안에 조건을 넣으면 모든 Document중 해당 조건을 만족하는 Document만 반환된다.

In [21]:
for doc in conn_db_collection.find( {'age' : 32 }):
    print(doc)

{'_id': ObjectId('5f414e5b84fc9297a70c5c23'), 'name': 'fox', 'age': 32}


---
- <b>`count_documents()`</b> : 조건에 맞는 Document 갯수 출력

In [24]:
conn_db_collection.count_documents({'age': 32})

1

---
- <b>`sort()`</b> : 오름차순 정렬

In [29]:
docs = conn_db_collection.find().sort('age')

for doc in docs:
    print(doc)

{'_id': ObjectId('5f3fcaf084fc9297a70c5c18'), 'Name': 'JihyunRyu', 'Age': 23, 'School': 'SeoulTech.Univ', 'Major': 'MSDE'}
{'_id': ObjectId('5f3fcde484fc9297a70c5c19'), 'Student ID': '17102020', 'Grade': 'Senior'}
{'_id': ObjectId('5f3fcde484fc9297a70c5c1a'), 'Github URL': 'https://github.com/jhryu1208', 'Github User_ID': 'jhryu1208'}
{'_id': ObjectId('5f414c1684fc9297a70c5c1b'), 'Naver': 'jhryu1208@naver.com', 'Google': 'jhryu1208@gmail.com'}
{'_id': ObjectId('5f414e2b84fc9297a70c5c1c'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('5f414e2b84fc9297a70c5c1d'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('5f414e5b84fc9297a70c5c24'), 'name': 'hmm'}
{'_id': ObjectId('5f414e5b84fc9297a70c5c1e'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('5f414e5b84fc9297a70c5c20'), 'name': 'cathy', 'age': 25}
{'_id': ObjectId('5f414e5b84fc9297a70c5c21'), 'name': 'david', 'age': 27}
{'_id': ObjectId(

<br>
<br>

### (3) Document Update(수정) ( update_one() 과 update_many() )
---
- <b>`update_one()`</b> : 가장 먼저 검색되는 한 종류의 Docuemnt만 수정하기

In [30]:
conn_db_collection.find_one()

{'_id': ObjectId('5f3fcaf084fc9297a70c5c18'),
 'Name': 'JihyunRyu',
 'Age': 23,
 'School': 'SeoulTech.Univ',
 'Major': 'MSDE'}

- <U>이때, $set 사용시, 없는 key를 삽입해서 데이터를 추가할 수 있다.</U>

In [40]:
conn_db_collection.update_one( {'Name' : 'JihyunRyu' },
    { '$set' : {'Major' : 'Manufacturing Systems and Design Engineering' } }
)

In [41]:
conn_db_collection.find_one()

{'_id': ObjectId('5f3fcaf084fc9297a70c5c18'),
 'Name': 'JihyunRyu',
 'Age': 23,
 'School': 'SeoulTech.Univ',
 'Major': 'Manufacturing Systems and Design Engineering'}

___
- <b>update_many()</b> : 조건에 맞는 <U>모든</U> Document 수정하기

In [45]:
for doc in conn_db_collection.find( {  'age' : {'$gt' : 23} } ):
    print(doc)

{'_id': ObjectId('5f414e5b84fc9297a70c5c1f'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('5f414e5b84fc9297a70c5c20'), 'name': 'cathy', 'age': 25}
{'_id': ObjectId('5f414e5b84fc9297a70c5c21'), 'name': 'david', 'age': 27}
{'_id': ObjectId('5f414e5b84fc9297a70c5c22'), 'name': 'erick', 'age': 28}
{'_id': ObjectId('5f414e5b84fc9297a70c5c23'), 'name': 'fox', 'age': 32}


In [47]:
conn_db_collection.update_many( { 'age' : { '$gt' : 23 } }, 
                               { '$set' : { 'name':'jhryu1208' } }
                              )

for doc in conn_db_collection.find( {  'age' : {'$gt' : 23} } ):
    print(doc)

{'_id': ObjectId('5f414e5b84fc9297a70c5c1f'), 'name': 'jhryu1208', 'age': 30}
{'_id': ObjectId('5f414e5b84fc9297a70c5c20'), 'name': 'jhryu1208', 'age': 25}
{'_id': ObjectId('5f414e5b84fc9297a70c5c21'), 'name': 'jhryu1208', 'age': 27}
{'_id': ObjectId('5f414e5b84fc9297a70c5c22'), 'name': 'jhryu1208', 'age': 28}
{'_id': ObjectId('5f414e5b84fc9297a70c5c23'), 'name': 'jhryu1208', 'age': 32}


<br>
<br>

### (4) Document Delete(삭제) ( delete_one() 과 delete_many() )
---
- <b>`delete_one()`</b> : 가장 먼저 검색되는 한가지 Document만 삭제하기

In [49]:
docs = conn_db_collection.find({'name' : 'jhryu1208'})

for doc in docs:
    print(doc)

{'_id': ObjectId('5f414e5b84fc9297a70c5c1f'), 'name': 'jhryu1208', 'age': 30}
{'_id': ObjectId('5f414e5b84fc9297a70c5c20'), 'name': 'jhryu1208', 'age': 25}
{'_id': ObjectId('5f414e5b84fc9297a70c5c21'), 'name': 'jhryu1208', 'age': 27}
{'_id': ObjectId('5f414e5b84fc9297a70c5c22'), 'name': 'jhryu1208', 'age': 28}
{'_id': ObjectId('5f414e5b84fc9297a70c5c23'), 'name': 'jhryu1208', 'age': 32}


In [51]:
conn_db_collection.delete_one({'name' : 'jhryu1208'})

# 맨 첫줄 Document가 삭제되었음을 확인할 수 있다.
docs = conn_db_collection.find({'name' : 'jhryu1208'})
for doc in docs:
    print(doc)

{'_id': ObjectId('5f414e5b84fc9297a70c5c21'), 'name': 'jhryu1208', 'age': 27}
{'_id': ObjectId('5f414e5b84fc9297a70c5c22'), 'name': 'jhryu1208', 'age': 28}
{'_id': ObjectId('5f414e5b84fc9297a70c5c23'), 'name': 'jhryu1208', 'age': 32}


---
- <b>`delete_many()`</b> : 조건에 맞는 모든 Document 삭제하기 

In [52]:
conn_db_collection.delete_many( {'name' : 'jhryu1208'} )

docs = conn_db_collection.find({'name' : 'jhryu1208'})
for doc in docs:
    print(doc)

In [53]:
# name이 jhryu1208인 모든 Document가 삭제되었음을 확인 할 수 있다.
conn_db_collection.count_documents({'name' : 'jhryu1208'})

0